In [1]:
import load_data 
import model_utils
import run_network
import network 
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2 as cv
import os
from torch import nn, optim
from torchvision import transforms, utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from skimage.measure import compare_ssim as ssim
import csv
%matplotlib inline

In [ ]:
path = './data/'
filename = 'dataset_batch_3.mat'
train_input_dataset,train_output_dataset = load_data.load_dataset(path,filename)
print(train_input_dataset.shape,train_output_dataset.shape)

In [ ]:
path = './data/'
filename = 'dataset_test.mat'
test_input_dataset,test_output_dataset = load_data.load_dataset(path,filename)
print(test_input_dataset.shape,test_output_dataset.shape)


In [ ]:
inputs_nums = [25,50,75,100,125,150,175,200]
batch_size = 80
device = torch.device("cuda:0")
for i in range(7,len(inputs_nums)):
    training_dataset = load_data.DeconvolutionDataset(train_input_dataset,train_output_dataset,
                                                  transform=transforms.Compose([load_data.ImageSelect(inputs_nums[i]),
                                                                              load_data.ToTensor()]))
    training_dataloader = DataLoader(training_dataset, batch_size=batch_size,shuffle=True, num_workers=6)

    
    if i == 0: 
        model = network.FullyConvNet_25()
    elif i == 1:
        model = network.FullyConvNet_50()
    elif i == 2:
        model = network.FullyConvNet_75()
    elif i == 3:
        model = network.FullyConvNet_100()
    elif i == 4:
        model = network.FullyConvNet_125()
    elif i == 5:
        model = network.FullyConvNet_150()
    elif i == 6:
        model = network.FullyConvNet_175()
    elif i == 7:
        model = network.FullyConvNet_200()
    
    model.to(device)

    criterion = nn.SmoothL1Loss()

    optimizer = optim.Adam(model.parameters(), lr=1e-6)
    num_epochs = 5000

    run_network.train(training_dataloader,model,criterion, optimizer, num_epochs,device)
    
    test_data = np.asarray(np.transpose(test_input_dataset[:4,:inputs_nums[i],:,:],(0,1,2,3)),dtype=np.float32)
    test_data = torch.from_numpy(test_data)
    
    predict_test = run_network.test(test_data,model,device)

    inputs = inputs_nums[i]
    input_num = str(inputs)

    save_path = './result/'
    save_filename = 'result_psnr_ssim.csv'
    psnr_value = ['psnr']
    ssim_value = ['ssim']
    psnr_value.append(inputs)
    ssim_value.append(inputs)
    run_network.save_result(save_path,input_num,psnr_value,ssim_value,predict_test,test_output_dataset,save_filename)

In [ ]:
# def train(train_loader,model,criterion, optimizer, num_epochs,device):
#     model.train()
#     for epoch in range(num_epochs):
#         for batch_idx, sample in enumerate(train_loader):
#             inputs,target = sample['input'].to(device),sample['output'].to(device)

#             # forward
#             out = model(inputs)
#             loss = criterion(out, target)
#             # backward
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#         if epoch < 100:
#             print('Epoch[{}/{}], loss: {:.6f}'
#                   .format(epoch+1, num_epochs, loss.data[0]))
#         elif (epoch+1) % 100 == 0:
#             print('Epoch[{}/{}], loss: {:.6f}'
#                   .format(epoch+1, num_epochs, loss.data[0]))

In [ ]:
# device = torch.device("cuda:0")
# model = network.FullyConvNet_25()
# # model = nn.DataParallel(model)
# # model.cuda()
# model.to(device)

# # criterion_2 = nn.MSELoss()
# criterion = nn.SmoothL1Loss()

# # optimizer = optim.SGD(model.parameters(), lr=1e-9)
# optimizer = optim.Adam(model.parameters(), lr=1e-6)
# num_epochs = 5000

In [ ]:
# train(training_dataloader,model,criterion, optimizer, num_epochs,device)

In [ ]:
# def test(test_data,model,device):
#     model.eval()
#     predict = model(test_data.to(device))
#     predict_test = predict.data.cpu().numpy()
#     return predict_test

In [ ]:
# """
# Video Quality Metrics
# Copyright (c) 2014 Alex Izvorski <aizvorski@gmail.com>

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
# """

# import numpy
# import math

# def psnr(img1, img2):
#     mse = numpy.mean( (img1 - img2) ** 2 )
#     if mse == 0:
#         return 100
#     PIXEL_MAX = 255.0
#     return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [ ]:
# predict_test = test(test_data,model,device)

In [ ]:
# def normalize_image(img):
#     return img/(img.max()-img.min())*255

In [ ]:
# inputs = 25
# input_num = str(inputs)

# save_path = './result/'
# save_filename = 'result_psnr_ssim.csv'
# psnr_value = ['psnr']
# ssim_value = ['ssim']
# psnr_value.append(inputs)
# ssim_value.append(inputs)
# def save_result(save_path,input_num,psnr_value,ssim_value,input_num):
#     filepath = os.path.join(save_path,input_num)
#     if not os.path.exists(filepath):
#         os.makedirs(filepath)
#     for i in range(4):
#         filename = 'test_predict_'+str(i)+'.png'
#         cv.imwrite(os.path.join(filepath,filename),normalize_image(predict_test[i,0,:,:]))
#         psnr_value.append('%.4f' % psnr(normalize_image(predict_test[i,0,:,:]),normalize_image(test_output_dataset[i,:,:])))
#         ssim_value.append('%.4f' % ssim(normalize_image(predict_test[i,0,:,:]),normalize_image(test_output_dataset[i,:,:]),data_range=255))
#     metric_value = [psnr_value,ssim_value]
#     if not os.path.isfile(os.path.join(save_path,save_filename)):
#         myfile = open(os.path.join(save_path,save_filename),'w')
#     else:
#         myfile = open(os.path.join(save_path,save_filename),'a')
#     with myfile:
#         writer = csv.writer(myfile)
#         writer.writerows(metric_value)
#     myfile.close()
        

In [ ]:
# inputs = 25
# save_path = './result/'
# input_num = str(inputs)
# if not os.path.exists(os.path.join(save_path,input_num)):
#         os.makedirs(os.path.join(save_path,input_num))

In [ ]:
# filepath = os.path.join(save_path,input_num)
# psnr_value = ['psnr']
# ssim_value = ['ssim']
# psnr_value.append(inputs)
# ssim_value.append(inputs)
# for i in range(4):
#     filename = 'test_predict_'+str(i)+'.png'
#     cv.imwrite(os.path.join(filepath,filename),normalize_image(predict_test[i,0,:,:]))
#     psnr_value.append('%.4f' % psnr(normalize_image(predict_test[i,0,:,:]),normalize_image(test_output_dataset[i,:,:])))
#     ssim_value.append('%.4f' % ssim(normalize_image(predict_test[i,0,:,:]),normalize_image(test_output_dataset[i,:,:]),data_range=255))
    
# metric_value = [psnr_value,ssim_value]
# save_filename = 'result_psnr_ssim.csv'
# if not os.path.isfile(os.path.join(save_path,save_filename)):
#     myfile = open(os.path.join(save_path,save_filename),'w')
# else:
#     myfile = open(os.path.join(save_path,save_filename),'a')
# with myfile:
#     writer = csv.writer(myfile)
#     writer.writerows(metric_value)
# myfile.close()

In [ ]:
# print(ssim_value)

In [ ]:
# plt.imshow(predict_test[1,0,:,:])
# predict_test[1,0,:,:].max()

In [ ]:
# plt.imshow(test_output_dataset[1,:,:])

In [ ]:
# psnr(predict_test[0,0,:,:]/np.max(predict_test[1,0,:,:]),test_output_dataset[0,:,:]/np.max(test_output_dataset[0,:,:]))

In [ ]:
# np.max(predict_test[1,0,:,:])

In [ ]:
# plt.imshow(test_output_dataset[1,:,:]-predict_test[1,0,:,:])

In [ ]:
# a = test_output_dataset[1,:,:]-predict_test[1,0,:,:]

In [ ]:
# np.max(abs(a))

In [ ]:
# ssim(test_output_dataset[1,:,:],predict_test[1,0,:,:],data_range=predict_test[1,0,:,:].max()-predict_test[1,0,:,:].min())

In [ ]:
# ssim(test_output_dataset[1,:,:],test_output_dataset[1,:,:],data_range=test_output_dataset[1,:,:].max()-test_output_dataset[1,:,:].min())

In [ ]:
# def mse(img1,img2):
#     mse = numpy.mean( (img1 - img2) ** 2 )
#     return mse

In [ ]:
# mse(predict_test[0,0,:,:],test_output_dataset[0,:,:])

In [ ]:
# def normalize_image(img):
#     return img/(img.max()-img.min())*255

In [ ]:
# pred = normalize_image(predict_test[1,0,:,:])
# plt.imshow(pred)

In [ ]:
# gt = normalize_image(test_output_dataset[1,:,:])
# plt.imshow(gt)

In [ ]:
# mse(gt,pred)

In [ ]:
# psnr(gt,pred)

In [ ]:
# ssim(gt,pred,data_range=255)